In [40]:
!pip install streamlit
!pip install lime
!pip3 install pydicom

In [41]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-04-28 03:21:49--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.129.29, 52.200.34.95, 34.232.92.41, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.129.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13828408 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.6’

ngrok-stable-linux- 100%[===================>]  13.19M  31.4MB/s    in 0.4s    

2021-04-28 03:21:50 (31.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.6’ saved [13828408/13828408]



In [43]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [44]:
get_ipython().system_raw('./ngrok http 8501 &')

In [45]:
%%writefile functions.py

import keras
import numpy as np
import streamlit as st
from keras import layers, models, optimizers
from PIL import Image
import pydicom
import cv2
from skimage import measure
from skimage.transform import resize

MODEL = "/content/drive/My Drive/ProjectDataset/Ioumodeltemp.h5"

@st.cache(allow_output_mutation=True)
def load_model():
    model = keras.models.load_model(MODEL, compile=True)
    return model

def preprocess_image(img):
    image = Image.open(img).convert("RGB")
    p_img = image.resize((128, 128))
    return np.array(p_img) / 255.0

def predict(model, img):
    prob = model.predict(np.reshape(img, [1, 128, 128, 3]))
    component = prob[:,:,0] > 0.5
    component = measure.label(component)
    
    confidence = 0
    for area in measure.regionprops(component):
      y, x, y2, x2 = area.bbox
      height = y2 - y
      width = x2 - x
      confidence = np.mean(prob[y:y+height, x:x+width])

    if confidence > 0.5:
        prediction = True
    else:
        prediction = False

    return prob, prediction

Overwriting functions.py


In [46]:
%%writefile ex.py

import streamlit as st
import numpy as np
import functions
from PIL import Image

img = st.file_uploader(label="Load X-Ray Chest image")

if img is not None:
    # Preprocessing Image
    p_img = functions.preprocess_image(img)

    if st.checkbox('Zoom image'):
      image = np.array(Image.open(img))
      st.image(image, use_column_width=True)
    else:
      st.image(p_img)

    # Loading model
    loading_msg = st.empty()
    loading_msg.text("Predicting...")
    model = functions.load_model()

    # Predicting result
    prob, prediction = functions.predict(model, p_img)

    if prediction:
        st.markdown(unsafe_allow_html=True, body="<span style='color:red; font-size: 50px'><strong><h4>Pneumonia!</h4></strong></span>")
    else:
        st.markdown(unsafe_allow_html=True, body="<span style='color:green; font-size: 50px'><strong><h3>Healthy!</h3></strong></span>")

Overwriting ex.py


In [47]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: http://c2cc0bfccd4f.ngrok.io


In [48]:
!streamlit run ex.py

2021-04-28 03:22:04.644787: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.86.229.37:8501

2021-04-28 03:22:12.049804: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-28 03:22:12.050959: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-28 03:22:12.063419: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-28 03:22:12.063476: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (29fb160dc190): /proc/driver/nvidia/version does not exist
2021-04-28 03:22:12.064436: I tensorflow/compiler/jit/xla_gpu_device.cc:99]